<a href="https://colab.research.google.com/github/BokyungQQ/Boot_camp_Final_project/blob/main/%EC%A0%84%EC%B2%98%EB%A6%AC%20%EB%B0%8F%20EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
plt.rc('font', family='NanumGothic')

In [ ]:
customers = pd.read_csv('/content/user.csv')
order = pd.read_csv('/content/order.csv')
review = pd.read_csv('/content/review.csv')
products = pd.read_csv('/content/상품 리스트.csv')

# 전처리

## order

- 상품명에 ‘당일출고’, ‘예약발송’ 등 문구가 붙어있는 경우 존재 ⇒ 정리
    - ‘배송유형’ 컬럼 생성 (0: 당일출고, 1: 예약발송, 2: 없음)
    - **‘상품명_옵션’** 컬럼에 ‘상품명_색상’ 정보만 들어가도록 정리 (+ 대괄호 제거)

In [ ]:
# unnamed 컬럼 제거
order = order.iloc[:,1:]
order

In [ ]:
order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11563 entries, 0 to 11562
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   주문일시            11563 non-null  object 
 1   주문번호            11563 non-null  object 
 2   회원등급            6533 non-null   object 
 3   총 결제금액          11563 non-null  float64
 4   결제수단            11563 non-null  object 
 5   사용한 적립금액(최종)    11563 non-null  float64
 6   상품명(어라운드앤 한국)   11563 non-null  object 
 7   주문자 주소          11555 non-null  object 
 8   주문자ID           11563 non-null  object 
 9   수령인 주소          11555 non-null  object 
 10  반품신청일           265 non-null    object 
 11  취소신청일           597 non-null    object 
 12  교환신청일           15 non-null     object 
 13  총 결제금액(KRW)     11563 non-null  int64  
 14  사용한 쿠폰명         5415 non-null   object 
 15  주문서 쿠폰 할인금액     5415 non-null   float64
 16  쿠폰 할인금액(최종)     11563 non-null  float64
 17  주문상품명           11563 non-null 

In [ ]:
# 당일출고와 예약발송의 경우 새로운 칼럼 생성
def assign_shipping_type(product_name):
    if '당일출고' in product_name:
        return 0
    elif '예약발송' in product_name:
        return 1
    else:
        return 2

In [ ]:
order['배송유형'] = order['상품명'].apply(assign_shipping_type)
order = order.rename(columns={'상품명': '상품명_옵션'})
order

In [ ]:
# 상품명에서 괄호 앞부분 지우기
def remove_prefix(product_name):
    if '[' in product_name:
        return re.sub(r'^[^\[]*', '', product_name).strip()
    else:
        return product_name

In [ ]:
order['상품명_옵션'] = order['상품명_옵션'].apply(remove_prefix)
order

In [ ]:
# 상품명에서 괄호도 모두 삭제
def remove_brackets(product_name):
    return re.sub(r'[\[\]]', '', product_name).strip()

order['상품명_옵션'] = order['상품명_옵션'].apply(remove_brackets)
order

- 상품별 비교를 위해 **‘상품명’** 컬럼 생성 (색상 정보 제거)
    - 모든 키링은 ‘keyring’ 범주에 포함
    - 기타 상품(wallet, shoes 등)은 ‘acc’ 범주에 포함
    - ‘gift_package’는 따로 범주 생성

In [ ]:
# '상품명_옵션'에서 색상 정보 제거
order['상품명'] = order['상품명_옵션'].apply(lambda x: x.split('_')[0])

In [ ]:
order.groupby('상품명').size()

In [ ]:
# 키링 -> 'keyring' 범주화
# 'acc' 범주화

for i in range(len(order)):
    if 'keyring' in order['상품명'][i]:
        order['상품명'][i] = 'keyring'
    elif 'Gift' in order['상품명'][i]:
        order['상품명'][i]
    elif 'bag' not in order['상품명'][i]:
        order['상품명'][i] = 'acc'
    else:
        order['상품명'][i]

order.groupby('상품명').size()

**이상데이터 제거**
- 배송지 없는 경우 (테스트용)
- 내부직원 아이디 삭제

In [ ]:
# 배송지가 없는 경우
order = order[~order['주문자 주소'].isnull()]

# 직원 ID 제외
exclude_id = ['2688120606@k', 'andugwak', '2300943014@k', 'yeriii94', '2832968400@k', '2296005899@k', '3115853248@k', 'jinju12', 'jinju1219', '2304150831@k', '2936163205@k', '2372723095@k']
order = order[~order['주문자ID'].isin(exclude_id)]

order

## order 주문상태 컬럼

In [ ]:
order['주문상태'] = ""

In [ ]:
# '총 결제금액'이 3000.0인 경우 '반품', 그렇지 않으면 '정상'으로 설정
order['주문상태'] = np.where(order['총 결제금액'] == 3000.0, '반품', '정상')

conditions = [
    (order['총 결제금액'] == 3000.0) | (order['반품신청일'].notnull()),
    ((order['결제수단'].isin(['무통장입금', '가상계좌'])) & (order['총 결제금액'] == 0.0)),
    (order['교환신청일'].notnull()),
    (((order['결제수단'].isin(['신용카드', '쿠폰,신용카드', '계좌이체'])) & (order['총 결제금액'] == 0.0)) | (order['취소신청일'].notnull()) | ((order['네이버 페이 취소접수 구분'].notnull()) & (order['교환신청일'].isnull())))
]
choices = ['반품', '미결제', '교환', '취소']
order['주문상태'] = np.select(conditions, choices, default='정상')

In [ ]:
order

In [ ]:
for i in range(len(order)):

    # 반품
    if order['총 결제금액'][i] == 3000.0:
        order['주문상태'][i] = '반품'
    elif order['반품신청일'][i] is not None:
        order['주문상태'][i] = '반품'

    # 미결제
    elif (order['결제수단'][i] in ['무통장입금','가상계좌']) & (order['총 결제금액'][i] == 0.0):
        order['주문상태'][i] = '미결제'

    # 교환
    elif order['교환신청일'][i] is not None:
        order['주문상태'][i] = '교환'

    # 취소
    elif (order['결제수단'][i] in ['신용카드','쿠폰,신용카드','계좌이체']) & (order['총 결제금액'][i] == 0.0):
        order['주문상태'][i] = '취소'
    elif order['취소신청일'][i] is not None:
        order['주문상태'][i] = '취소'
    elif (order['네이버 페이 취소접수 구분'] is not None) & (order['교환신청일'] is None):
        order['주문상태'][i] = '취소'

    # 정상
    else:
        order['주문상태'][i] = '정상'

## order_buy

**판매건/매출액 등을 활용해 분석할 경우, 취소/반품/미결제 건 제외 필요 : order_buy**


[취소]

- ‘취소신청일’에 날짜 데이터 존재하는 경우
- **신용카드 / 쿠폰,신용카드 / 계좌이체 & 결제금액 0원**인 경우
- '네이버 페이 취소접수 구분'에 메세지 있는 경우 (교환 제외)

[미결제]

- **무통장입금 / 가상계좌 & 결제금액 0원**인 경우

[반품]

- ‘반품신청일’에 날짜 데이터 존재하는 경우
- 결제금액이 **3000원, 6000원**인 경우

In [ ]:
order.shape

In [ ]:
# 반품 건수, 취소 건수
print("반품: ", order['반품신청일'].notnull().sum())
print("취소: ", order['취소신청일'].notnull().sum())

In [ ]:
# 반품 제거
order_buy = order.drop(order[order['반품신청일'].notnull()].index, axis=0)

# 반품 배송비만 결제한 경우 삭제
order_buy = order_buy[order_buy['총 결제금액'] != 3000]
order_buy = order_buy[order_buy['총 결제금액'] != 6000]
order_buy

In [ ]:
# 미결제 제거
order_buy = order_buy.drop(order_buy[(order_buy['결제수단'].isin(['무통장입금','가상계좌'])) & (order_buy['총 결제금액']==0.0)].index, axis=0)

order_buy.shape

In [ ]:
# 취소 제거
order_buy = order_buy.drop(order_buy[order_buy['취소신청일'].notnull()].index, axis=0)

# 신용카드, 계좌이체, 쿠폰/신용카드
order_buy = order_buy.drop(order_buy[(order_buy['결제수단'].isin(['신용카드','계좌이체','쿠폰,신용카드'])) & (order_buy['총 결제금액']==0.0)].index, axis=0)

# '네이버 페이 취소접수 구분'에 메세지 있는 경우 제외 (교환 제외)
order_buy = order_buy.drop(order_buy[(order_buy['네이버 페이 취소접수 구분'].notnull()) & (order_buy['교환신청일'].isnull())].index, axis=0)

order_buy.shape

In [ ]:
#order_buy.to_csv('./order_buy_1205.csv')

## 매출액 확인 위한 데이터셋 생성
- 매출액 분석하는 경우, 중복되는 주문번호 제거
    - 한 번 주문에 여러 상품을 구매하는 경우, 행이 상품의 개수만큼 생성되고 각 행의 결제 금액에 합산 금액이 표시되기 때문

In [ ]:
order_sales = order_buy.drop_duplicates(['주문번호'])

In [ ]:
# order_sales.to_csv('./order_sales.csv')

# EDA

## 단품/세트 구매율

In [ ]:
order_buy

In [ ]:
# 카테고리 생성
def category(product_name):
    if 'bag' in product_name:
        return 'bag'
    else:
        return product_name

In [ ]:
order_buy['카테고리'] = order_buy['상품명'].apply(category)

In [ ]:
order_buy

In [ ]:
# 단품 구매 / 세트 구매 구분
# 주문번호가 중복되는 경우 : 여러 상품 세트 구매
# 주문번호가 한 행인 경우 : 단품 구매 (수량 무관)

onething = order_buy[~order_buy['주문번호'].duplicated(keep=False)]
sets = order_buy[order_buy['주문번호'].duplicated(keep=False)]

In [ ]:
# 단품 구매 : 8250건
onething

In [ ]:
# 세트 구매
sets

In [ ]:
# 정렬
sets = sets.sort_values(by=['주문일시','주문번호','카테고리'])

# 주문별 구매한 카테고리 추출
# 세트 구매 : 978건
sets_cat = sets.groupby('주문번호').agg({'카테고리':lambda x: ', '.join(set(x))}).reset_index()
sets_cat

In [ ]:
sets_cat.groupby('카테고리').size()

In [ ]:
8250+978

In [ ]:
# 단품 구매한 주문 : 8250건
# 세트 구매한 주문 : 978건
# 전체 주문(개별) : 9228건

print('단품구매율: ', round(8250/9228*100, 1),'%')
print('세트구매율: ', round(978/9228*100, 1),'%')

In [ ]:
Final_buy['주문번호'].nunique()

## 상품별 리뷰 확인

### 리뷰 오더 조인

In [ ]:
review = review.iloc[:,1:]
review

In [ ]:
review['new_주문번호'] = review['주문번호'].str.split('-').str.get(0) + '-' + review['주문번호'].str.split('-').str.get(1)

In [ ]:
review

In [ ]:
# 상품명에서 괄호 앞부분 지우기
def remove_prefix(product_name):
    if '[' in product_name:
        return re.sub(r'^[^\[]*', '', product_name).strip()
    else:
        return product_name
# 상품명에서 괄호도 모두 삭제
def remove_brackets(product_name):
    return re.sub(r'[\[\]]', '', product_name).strip()

In [ ]:
# 상품명 정리 => 상품명_옵션
review['상품명'] = review['상품명'].apply(remove_prefix)
review['상품명'] = review['상품명'].apply(remove_brackets)

review = review.rename(columns={'상품명': '상품명_옵션'})

In [ ]:
# 상품명 => 색상 정보 제거
review['상품명'] = review['상품명_옵션'].apply(lambda x: x.split('_')[0])

In [ ]:
# 키링 => keyring
for i in range(len(review)):
    if 'keyring' in review['상품명'][i]:
        review['상품명'][i] = 'keyring'
    else:
        review['상품명'][i]

review.groupby('상품명').size()

In [ ]:
review.to_csv('./review.csv')

In [ ]:
# review 조인
Final_BR = pd.merge(Final_buy, review, how='inner', left_on=['주문번호','상품명_옵션'], right_on=['new_주문번호','상품명_옵션'])
Final_BR

In [ ]:
Final_BR.info()

In [ ]:
Final_BR = Final_BR.rename(columns={'상품명_x': '상품명'})
Final_BR = Final_BR.rename(columns={'상품명_y': '상품명_review'})

### 리뷰 분석

In [ ]:
# 각 상품별 리뷰 수
cnt_review = Final_BR.groupby('상품명').size().reset_index(name='cnt_review')
cnt_review

In [ ]:
# 각 상품별 주문 수
cnt_order = Final_buy.groupby('상품명').size().reset_index(name='cnt_order')
cnt_order

In [ ]:
cnt = pd.merge(cnt_order, cnt_review, how='inner', on='상품명')
cnt

In [ ]:
# 구매 건수 대비 리뷰 작성 비율 구하기
cnt['rate'] = round(cnt['cnt_review']/cnt['cnt_order']*100, 1)
cnt.sort_values(by='rate', ascending=False)

In [ ]:
mean_score = Final_BR.groupby('상품명')['리뷰평점'].mean().reset_index(name='평균평점')
mean_score

In [ ]:
review_cnt = pd.merge(cnt, mean_score, on='상품명', how='inner')
review_cnt

In [ ]:
review_cnt[(review_cnt['cnt_order']>=50) & (review_cnt['cnt_review']>=10)].sort_values(by='cnt_order', ascending=False)

In [ ]:
# review_cnt.to_csv('./cnt.csv')